In [1]:
import os 
import numpy as np
import matplotlib.pylab as plt
from itertools import zip_longest
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import json
import collections
import phycv
from phycv import PST, PST_GPU,VEVID,VEVID_GPU
import matplotlib.image as mpimg
from PIL import Image
from io import BytesIO
import cv2
import tqdm
import torchvision.transforms as transforms
import copy
from PIL import Image

In [1]:
class DataProcesser:
    def __init__(self,input_directory="../bdd100k/images/100k/",output_directory="../bdd100k_preprocessed/images/",labels_path = '../bdd100k/labels/',train=True,mode=2):
        if train:
            self.mode = "train"
        else:
            self.mode = "val"
        self.input_directory=os.path.join(input_directory,self.mode)
        self.output_directory=os.path.join(output_directory,self.mode)
        self.labels_path=labels_path
        if mode==2:
            self.df = self.open_dataframe2()
        else:
            self.df = self.open_dataframe()
    
    def process_images(self,mode=2):
        def process(name):
            if mode==2:
                return name+".jpg"
            else:
                return name
        def helper(name,time):
            if time=="night":
                self.process_image_pst_vevid(os.path.join(self.input_directory,name),self.output_directory,name)
            else:
                self.process_image_pst_vevid(os.path.join(self.input_directory,name),self.output_directory,name,b=None,G=None)
        self.df.apply(lambda x: helper(process(x["name"]), x["attributes.timeofday"]), axis=1)
    
    def open_dataframe(self):
        with open(self.labels_path+'bdd100k_labels_images_'+self.mode+'.json') as data_file:
            data = json.load(data_file)
        df = pd.json_normalize(data)
        return df
    
    def open_dataframe2(self):
        data=[]
        path = os.path.join(self.labels_path,self.mode)
        for f in os.listdir(path):
            with open(os.path.join(path,f)) as data_file:
                data_ = json.load(data_file)
            data.append(data_)
        df = pd.json_normalize(data)
        return df
    
    @staticmethod
    def process_image_pst_vevid(img_file=None,output_path=None,output_name=None,S=0.4, W=20, sigma_LPF=0.1, thresh_min=0.0,thresh_max=0.8,morph_flag=1,b = 0.2, G = 10):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if b is not None:
            assert(G is not None)
            vevid_gpu = VEVID_GPU(device=device)
            original_image = vevid_gpu.run_lite(img_file,b,G)
        else:
            original_image = transforms.ToTensor()(mpimg.imread(img_file))
        if output_path:
            if not os.path.exists(output_path):
                os.makedirs(output_path)
        pst_gpu = PST_GPU(device=device)
        pst_gpu.load_img(img_array=original_image)
        pst_gpu.init_kernel(S,W)
        pst_gpu.apply_kernel(sigma_LPF,thresh_min,thresh_max,morph_flag)
        pst_output_gpu_torch=pst_gpu.pst_output
        pst_output_gpu = pst_output_gpu_torch.cpu().numpy()
        pst_gpu_result = Image.fromarray((pst_output_gpu * 255).astype(np.uint8))
        if output_path:
            pst_gpu_result.save(os.path.join(output_path, output_name))
        return pst_gpu_result

In [4]:
#Change labels path to where the label json files are (not including train/ or val/)
train_processer = DataProcesser(train=True,labels_path='G:/UCLA/Lane Detection Project/Lane Detection/bdd100k_preprocessed/det_annotations/data2/zwt/bdd/bdd100k/labels/100k')
train_processer.process_images()

C:\Users\PSH\anaconda3\envs\LaneDetection\Lib\site-packages\torchvision\transforms\functional.py:153: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:212.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


In [ ]:
val_processer = DataProcesser(train=False,labels_path='./data2/zwt/bdd/bdd100k/labels/100k')
val_processer.process_images()